In [1]:
import numpy as np
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from copy import deepcopy
import math
from minimol import Minimol
from tdc.benchmark_group import admet_group

/Users/manasmahale/miniconda3/envs/minimol/lib/python3.11/site-packages/minimol/model.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
class TaskHead(nn.Module):
    def __init__(self, hidden_dim=512, input_dim=512, dropout=0.1, depth=3, combine=True):
        super(TaskHead, self).__init__()
        self.dense1 = nn.Linear(input_dim, hidden_dim)
        self.dense2 = nn.Linear(hidden_dim, hidden_dim)
        self.dense3 = nn.Linear(hidden_dim, hidden_dim)
        self.final_dense = nn.Linear(input_dim + hidden_dim, 1) if combine else nn.Linear(hidden_dim, 1)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.bn3 = nn.BatchNorm1d(hidden_dim)
        self.dropout = nn.Dropout(dropout)
        self.combine = combine
        self.depth = depth

    def forward(self, x):
        original_x = x
        x = self.dense1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.dense2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout(x)
        if self.depth == 4:
            x = self.dense3(x)
            x = self.bn3(x)
            x = F.relu(x)
            x = self.dropout(x)
        x = torch.cat((x, original_x), dim=1) if self.combine else x
        x = self.final_dense(x)
        return x

In [3]:
class AdmetDataset(Dataset):
    def __init__(self, embeddings, targets):
        self.samples = embeddings
        self.targets = [float(t) for t in targets]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return torch.tensor(self.samples[idx]), torch.tensor(self.targets[idx])

In [4]:
def train_one_epoch(predictor, train_loader, optimiser, lr_scheduler, loss_fn, epoch, epochs, warmup):
    predictor.train()
    train_loss = 0
    if epoch < warmup: 
        lr_factor = epoch / warmup
    else: 
        lr_factor = (1 + math.cos(math.pi * (epoch - warmup) / (epochs - warmup))) / 2
    
    for param_group in optimiser.param_groups:
        param_group['lr'] = param_group['initial_lr'] * lr_factor

    for inputs, targets in train_loader:
        optimiser.zero_grad()
        logits = predictor(inputs).squeeze()
        loss = loss_fn(torch.sigmoid(logits), targets)
        loss.backward()
        optimiser.step()
        train_loss += loss.item()
    return predictor

In [5]:
benchmark_config = {
    'hia_hou': ('binary', False),
    'pgp_broccatelli': ('binary', False),
    'bioavailability_ma': ('binary', False),
    'bbb_martins': ('binary', False),
    'cyp2c9_veith': ('binary', False),
    'cyp2d6_veith': ('binary', False),
    'cyp3a4_veith': ('binary', False),
    'cyp2c9_substrate_carbonmangels': ('binary', False),
    'cyp2d6_substrate_carbonmangels': ('binary', False),
    'cyp3a4_substrate_carbonmangels': ('binary', False),
    'herg': ('binary', False),
    'ames': ('binary', False),
    'dili': ('binary', False)
}

In [6]:
# Hyperparameters from Minimol repo
SWEEP_RESULTS = {    
    'caco2_wang':                       {'hidden_dim': 2048, 'depth': 4, 'combine': True, 'lr': 0.0005},
    'hia_hou':                          {'hidden_dim': 2048, 'depth': 4, 'combine': True, 'lr': 0.0003},
    'pgp_broccatelli':                  {'hidden_dim': 512,  'depth': 4, 'combine': True, 'lr': 0.0003},
    'bioavailability_ma':               {'hidden_dim': 512,  'depth': 3, 'combine': True, 'lr': 0.0003},
    'lipophilicity_astrazeneca':        {'hidden_dim': 2048, 'depth': 4, 'combine': True, 'lr': 0.0005},
    'solubility_aqsoldb':               {'hidden_dim': 1024, 'depth': 4, 'combine': True, 'lr': 0.0005},
    'bbb_martins':                      {'hidden_dim': 2048, 'depth': 3, 'combine': True, 'lr': 0.0001},
    'ppbr_az':                          {'hidden_dim': 2048, 'depth': 4, 'combine': True, 'lr': 0.0003},
    'vdss_lombardo':                    {'hidden_dim': 1024, 'depth': 4, 'combine': True, 'lr': 0.0001},
    'cyp2d6_veith':                     {'hidden_dim': 512,  'depth': 3, 'combine': True, 'lr': 0.0001},
    'cyp3a4_veith':                     {'hidden_dim': 512,  'depth': 3, 'combine': True, 'lr': 0.0001},
    'cyp2c9_veith':                     {'hidden_dim': 512,  'depth': 3, 'combine': True, 'lr': 0.0001},
    'cyp2d6_substrate_carbonmangels':   {'hidden_dim': 512,  'depth': 3, 'combine': True, 'lr': 0.0001},
    'cyp3a4_substrate_carbonmangels':   {'hidden_dim': 512,  'depth': 3, 'combine': True, 'lr': 0.0001},
    'cyp2c9_substrate_carbonmangels':   {'hidden_dim': 1024, 'depth': 3, 'combine': True, 'lr': 0.0005},
    'half_life_obach':                  {'hidden_dim': 1024, 'depth': 3, 'combine': True, 'lr': 0.0003},
    'clearance_microsome_az':           {'hidden_dim': 1024, 'depth': 4, 'combine': True, 'lr': 0.0005},
    'clearance_hepatocyte_az':          {'hidden_dim': 2048, 'depth': 4, 'combine': True, 'lr': 0.0005},
    'herg':                             {'hidden_dim': 512,  'depth': 3, 'combine': True, 'lr': 0.0003},
    'ames':                             {'hidden_dim': 512,  'depth': 3, 'combine': True, 'lr': 0.0001},
    'dili':                             {'hidden_dim': 512,  'depth': 4, 'combine': True, 'lr': 0.0005},
    'ld50_zhu':                         {'hidden_dim': 1024, 'depth': 4, 'combine': True, 'lr': 0.0001},
}
# Default params if not in SWEEP
DEFAULT_PARAMS = {'hidden_dim': 512, 'depth': 3, 'combine': True, 'lr': 0.0003}

In [7]:
group = admet_group(path='data/')
featuriser = Minimol()
predictions = {}

Found local copy...


In [8]:
device = torch.device('cpu')

In [9]:
for admet_benchmark in tqdm(benchmark_config.keys()):
    benchmark = group.get(admet_benchmark)
    name = benchmark['name']
    train_df, test_df = benchmark['train_val'], benchmark['test']

    X_train = featuriser(list(train_df['Drug']))
    X_test = featuriser(list(test_df['Drug']))

    train_dset = AdmetDataset(X_train, train_df['Y'].values)
    test_dset = AdmetDataset(X_test, test_df['Y'].values)
    
    train_loader = DataLoader(train_dset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dset, batch_size=128, shuffle=False)

    params = SWEEP_RESULTS.get(name, DEFAULT_PARAMS)
    model = TaskHead(
        hidden_dim=params['hidden_dim'], 
        depth=params['depth'], 
        combine=params['combine']
    ).to(device)
    
    lr = params['lr']
    optimiser = optim.Adam(model.parameters(), lr=lr, weight_decay=0.0001)
    for pg in optimiser.param_groups: pg['initial_lr'] = lr
        
    loss_fn = nn.BCELoss()
    
    EPOCHS = 25
    WARMUP = 5
    
    for epoch in range(EPOCHS):
        model = train_one_epoch(model, train_loader, optimiser, None, loss_fn, epoch, EPOCHS, WARMUP)

    model.eval()
    all_preds = []
    with torch.no_grad():
        for inputs, _ in test_loader:
            logits = model(inputs).squeeze()
            probs = torch.sigmoid(logits)
            all_preds.extend(probs.cpu().numpy())
            
    predictions[f"{name}_y_pred_proba"] = np.array(all_preds)
    predictions[f"{name}_y_true"] = test_df['Y'].values

  0%|                                                                                          | 0/5 [00:00<?, ?it/s]

featurizing_smiles, batch=12:   0%|          | 0/8 [00:00<?, ?it/s]

/Users/manasmahale/miniconda3/envs/minimol/lib/python3.11/site-packages/torchmetrics/utilities/imports.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
/Users/manasmahale/miniconda3/envs/minimol/lib/python3.11/site-packages/torchmetrics/utilities/imports.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
/Users/manasmahale/miniconda3/envs/minimol/lib/python3.11/site-packages/torchmetrics/utilities/imports.py:23: UserWarning: pkg_resources is deprecated as an API. 

featurizing_smiles, batch=12:   0%|          | 0/8 [00:00<?, ?it/s]


 40%|████████████████████████████████▊                                                 | 2/5 [00:13<00:16,  5.60s/it]

...


 75%|█████████████████████████████████████████████████████████████▌                    | 3/4 [00:01<00:00,  2.24it/s]

featurizing_smiles, batch=9:   0%|          | 0/8 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.32it/s]

  0%|                                                                                          | 0/1 [00:00<?, ?it/s]

featurizing_smiles, batch=12:   0%|          | 0/8 [00:00<?, ?it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 13/13 [05:53<00:00, 27.21s/it]


In [10]:
with open('minimol_binary.pickle', 'wb') as handle:
    pickle.dump(predictions, handle, protocol=pickle.HIGHEST_PROTOCOL)